# Planet — Corporate Hybrid Capacity Forecast **v10**
## Stability & Regime Framework (Dept Monthly) + Risk-based Staffing + DOW Daily Plan + Real Productivity FTE

**Generated:** 2026-02-13 (Europe/Madrid)

### What’s new vs v9.x
- **Stability & regime classifier** per department (Stable / Volatile / Broken / LowData).
- **Model governance**: rejects non-converged ARIMA fits (prevents “poisoned blends”).
- **Adaptive strategy**
  - Stable → blend (Prophet/ARIMA/ETS) + PI
  - Volatile → conservative (Prophet+ETS, no blend)
  - Broken → regime window + Prophet (higher changepoints), ARIMA off
  - LowData → NaiveMean + mandatory risk uplift
- **Risk-based staffing policy** for critical verticals (Payments, Hospitality) + stability uplifts.

---


## 1) Setup (interactive BASE_DIR)

In [1]:
import os
from pathlib import Path
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

BASE_DIR = filedialog.askdirectory(title="Select BASE_DIR (CAPACITY folder)")
if not BASE_DIR:
    raise SystemExit("No folder selected. Execution stopped.")

BASE_DIR = str(Path(BASE_DIR).expanduser().resolve())
INPUT_DIR  = str(Path(BASE_DIR) / "input_model")
OUTPUT_DIR = str(Path(BASE_DIR) / "outputs")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

INCOMING_SOURCE_PATH = str(Path(INPUT_DIR) / "Incoming_new.xlsx")
INCOMING_SHEET = "Main"

DEPT_MAP_PATH = str(Path(INPUT_DIR) / "department.xlsx")
DEPT_MAP_SHEET = "map"

PRODUCTIVITY_PATH = str(Path(INPUT_DIR) / "productivity_agents.xlsx")
CASE_REASON_PATH = str(Path(INPUT_DIR) / "case_reason.xlsx")

OUTPUT_XLSX = str(Path(OUTPUT_DIR) / "capacity_forecast_hybrid_v10.xlsx")

# Horizons
H_MONTHS = 12
DAILY_HORIZON_DAYS = 90

SUPPORTED_LANGS = ["Spanish","English","Portuguese","French","German","Italian"]

# Prediction intervals
ENABLE_MONTHLY_PI = True
PI_ALPHA = 0.05  # 95% PI

# Backtesting config
ENABLE_DEPT_ACCURACY_TABLE = True
ACCURACY_BACKTEST_MONTHS = 9
ACCURACY_MIN_TRAIN_MONTHS = 12
ACCURACY_HORIZON_MONTHS = 1
ACCURACY_MAX_SPLITS = 9

# Bias recalibration
BIAS_RECAL_THRESHOLD_PCT = 10.0

# Critical verticals
CRITICAL_VERTICALS = {"Payments","Hospitality"}

# Stability thresholds (governance)
STAB = {
    "low_data_min_months": 18,
    "broken_z_thresh": 2.5,
    "volatility_cv_thresh": 0.65,
    "zero_share_thresh": 0.25,
    "broken_bias_thresh": 80.0,
    "broken_smape_thresh": 120.0,
}

# Staffing risk policy
RISK_POLICY = {
    "critical_low_acc_use_p95": 75.0,
    "critical_mid_acc_blend": 88.0,
    "volatility_uplift_pct": 10.0,
    "lowdata_uplift_pct": 15.0,
    "broken_uplift_pct": 20.0,
}

# DOW profile
DOW_LOOKBACK_DAYS = 180
DOW_MIN_OBS = 30

required_files = [INCOMING_SOURCE_PATH, DEPT_MAP_PATH, PRODUCTIVITY_PATH, CASE_REASON_PATH]
missing = [p for p in required_files if not Path(p).exists()]
if missing:
    raise FileNotFoundError("Missing required input files:\n- " + "\n- ".join(missing) + f"\n\nSelected BASE_DIR:\n{BASE_DIR}")

print("✅ Setup OK")
print("BASE_DIR:", BASE_DIR)
print("OUTPUT_XLSX:", OUTPUT_XLSX)


✅ Setup OK
BASE_DIR: C:\Projects\Capacity_forecast_2026
OUTPUT_XLSX: C:\Projects\Capacity_forecast_2026\outputs\capacity_forecast_hybrid_v10.xlsx


## 2) Imports

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import math
from typing import Optional, Dict, Tuple
from datetime import date, timedelta

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

try:
    from prophet import Prophet
except Exception:
    try:
        from fbprophet import Prophet
    except Exception:
        Prophet = None

print("Imports loaded. Prophet:", bool(Prophet))


Imports loaded. Prophet: True


## 3) Core functions (v10)

In [3]:
# ============================================================
# CORE FUNCTIONS — v10
# ============================================================

def smape(y_true, y_pred) -> float:
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom[denom == 0] = 1.0
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)

def clamp_nonneg(a):
    a = np.array(a, dtype=float)
    a[~np.isfinite(a)] = 0.0
    return np.clip(a, 0.0, None)

def _z_from_alpha(alpha: float) -> float:
    if abs(alpha - 0.10) < 1e-9: return 1.6448536269514722
    if abs(alpha - 0.05) < 1e-9: return 1.959963984540054
    if abs(alpha - 0.01) < 1e-9: return 2.5758293035489004
    return 1.959963984540054

def expm1_safe(x, cap_original: Optional[float] = None):
    a = np.array(x, dtype=float)
    a[~np.isfinite(a)] = -50.0
    a = np.maximum(a, -50.0)
    if cap_original and np.isfinite(cap_original) and cap_original > 0:
        log_cap = np.log1p(cap_original)
        a = np.minimum(a, log_cap)
    y = np.expm1(a)
    if cap_original and np.isfinite(cap_original) and cap_original > 0:
        y = np.minimum(y, cap_original)
    return np.clip(y, 0, None)

def compute_dynamic_cap(ts_m: pd.Series) -> float:
    if ts_m.empty or (ts_m.max() <= 0):
        return np.inf
    m12 = float(ts_m.tail(12).mean()) if len(ts_m) >= 3 else float(ts_m.mean())
    med, mx = float(ts_m.median()), float(ts_m.max())
    base = max(1.0, m12, med, 1.1 * mx)
    return base * 6.0

def compute_pi_from_log_mean_std(mean_log: np.ndarray, std_log: np.ndarray, cap: float, alpha: float = 0.05):
    z = _z_from_alpha(alpha)
    std_log = np.clip(np.array(std_log, dtype=float), 1e-8, None)
    lo_log = mean_log - z * std_log
    hi_log = mean_log + z * std_log
    mean = expm1_safe(mean_log, cap_original=cap)
    lo   = expm1_safe(lo_log,   cap_original=cap)
    hi   = expm1_safe(hi_log,   cap_original=cap)
    mean = clamp_nonneg(mean); lo = clamp_nonneg(lo); hi = clamp_nonneg(hi)
    return mean, np.minimum(lo, hi), np.maximum(lo, hi)

def coverage_95(actual: np.ndarray, lo: np.ndarray, hi: np.ndarray) -> float:
    a = np.array(actual, dtype=float)
    l = np.array(lo, dtype=float)
    h = np.array(hi, dtype=float)
    m = np.isfinite(a) & np.isfinite(l) & np.isfinite(h)
    if m.sum() == 0:
        return np.nan
    ok = (a[m] >= l[m]) & (a[m] <= h[m])
    return float(ok.mean() * 100.0)

def clean_outliers_daily(g: pd.DataFrame) -> pd.DataFrame:
    g = g.copy()
    s = g.sort_values("Date")["ticket_total"].astype(float)
    ql = s.quantile(0.01)
    qh = s.quantile(0.99)
    g["ticket_total"] = s.clip(ql, qh).values
    return g

def winsorize_monthly(ts_m: pd.Series, lower_q=0.01, upper_q=0.99) -> pd.Series:
    if ts_m.empty:
        return ts_m
    return ts_m.clip(lower=ts_m.quantile(lower_q), upper=ts_m.quantile(upper_q))

# ---------- Loaders ----------
def load_incoming(path: str, sheet_name: str) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet_name, engine="openpyxl")
    required = {"Date", "department_id"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Incoming missing columns: {sorted(list(missing))}. Found: {list(df.columns)}")
    if "ticket_total" not in df.columns:
        candidates = [c for c in ["total_incoming","incoming_total","Incoming"] if c in df.columns]
        if candidates:
            df["ticket_total"] = pd.to_numeric(df[candidates[0]], errors="coerce").fillna(0.0)
        elif {"incoming_from_customers","incoming_from_transfers"}.issubset(df.columns):
            df["ticket_total"] = (
                pd.to_numeric(df["incoming_from_customers"], errors="coerce").fillna(0.0)
                + pd.to_numeric(df["incoming_from_transfers"], errors="coerce").fillna(0.0)
            )
        else:
            raise ValueError("Incoming must contain ticket_total or known alternative columns.")
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])
    df["department_id"] = df["department_id"].astype(str).str.strip()
    df["ticket_total"] = pd.to_numeric(df["ticket_total"], errors="coerce").fillna(0.0).astype(float)
    if "language" not in df.columns:
        df["language"] = "English"
    df["language"] = df["language"].astype(str).str.strip()
    df["language"] = np.where(df["language"].isin(SUPPORTED_LANGS), df["language"], "English")
    if "department_name" not in df.columns:
        df["department_name"] = None
    if "vertical" not in df.columns:
        df["vertical"] = None
    return df

def load_dept_map(path: str, sheet: str) -> pd.DataFrame:
    mp = pd.read_excel(path, sheet_name=sheet, engine="openpyxl")
    rename_map = {"dept_id":"department_id","dept_name":"department_name","name":"department_name",
                  "segment":"vertical","vertical_name":"vertical"}
    mp = mp.rename(columns={k:v for k,v in rename_map.items() if k in mp.columns})
    if "department_id" not in mp.columns:
        raise ValueError(f"Department map must contain department_id. Found: {list(mp.columns)}")
    mp["department_id"] = mp["department_id"].astype(str).str.strip()
    if "department_name" in mp.columns:
        mp["department_name"] = mp["department_name"].astype(str).str.strip()
    if "vertical" in mp.columns:
        mp["vertical"] = mp["vertical"].astype(str).str.strip()
    cols = [c for c in ["department_id","department_name","vertical"] if c in mp.columns]
    return mp[cols].drop_duplicates("department_id")

def apply_mapping(incoming: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    df = incoming.merge(mapping, on="department_id", how="left", suffixes=("", "_map"))
    if "department_name_map" in df.columns:
        df["department_name"] = df["department_name"].fillna(df["department_name_map"])
    if "vertical_map" in df.columns:
        df["vertical"] = df["vertical"].fillna(df["vertical_map"])
    df["department_name"] = df["department_name"].fillna("Unknown")
    df["vertical"] = df["vertical"].fillna("Unmapped")
    df.drop(columns=[c for c in df.columns if c.endswith("_map")], inplace=True, errors="ignore")
    return df

def load_productivity(path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    df = pd.read_excel(path, engine="openpyxl")
    req = {"Date","agent_id","department_id","prod_total_model"}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"productivity missing columns: {sorted(list(missing))}. Found: {list(df.columns)}")
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])
    df["department_id"] = df["department_id"].astype(str).str.strip()
    df["prod_total_model"] = pd.to_numeric(df["prod_total_model"], errors="coerce")
    df = df[np.isfinite(df["prod_total_model"]) & (df["prod_total_model"] >= 0)].copy()
    df["dow"] = df["Date"].dt.weekday.astype(int)
    dept_prod = (df.groupby("department_id", as_index=False)["prod_total_model"]
                   .mean()
                   .rename(columns={"prod_total_model":"avg_tickets_per_agent_day"}))
    dept_dow_prod = (df.groupby(["department_id","dow"], as_index=False)["prod_total_model"]
                       .mean()
                       .rename(columns={"prod_total_model":"avg_tickets_per_agent_day_dow"}))
    return dept_prod, dept_dow_prod

def load_case_reason_outage_monthly(path: str) -> pd.DataFrame:
    df = pd.read_excel(path, engine="openpyxl")
    cols = {c.lower(): c for c in df.columns}
    date_col = next((cols[k] for k in ["date","created_date","created","opened","opened_date"] if k in cols), None)
    dept_col = next((cols[k] for k in ["department_id","dept_id","department"] if k in cols), None)
    reason_col = next((cols[k] for k in ["case_reason","reason","category","case_reason_name"] if k in cols), None)
    if date_col is None or dept_col is None or reason_col is None:
        raise ValueError(f"case_reason.xlsx missing required columns. Found: {list(df.columns)}")
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col])
    df[dept_col] = df[dept_col].astype(str).str.strip()
    df[reason_col] = df[reason_col].astype(str).str.strip()
    outage = df[df[reason_col].str.lower() == "global outage reported"].copy()
    outage["month"] = outage[date_col].dt.to_period("M")
    out = outage.groupby([dept_col,"month"], as_index=False).size().rename(columns={dept_col:"department_id","size":"outage_tickets"})
    out["department_id"] = out["department_id"].astype(str).str.strip()
    out["month"] = pd.PeriodIndex(out["month"], freq="M")
    return out

# ---------- Exogenous calendar ----------
def _easter_sunday(year: int) -> date:
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19*a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    L = (32 + 2*e + 2*i - h - k) % 7
    m = (a + 11*h + 22*L) // 451
    month = (h + L - 7*m + 114) // 31
    day = ((h + L - 7*m + 114) % 31) + 1
    return date(year, month, day)

def _last_friday_of_november(year: int) -> date:
    d = date(year, 11, 30)
    while d.weekday() != 4:
        d = d.replace(day=d.day - 1)
    return d

def build_eu_core_holidays(start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.DataFrame:
    years = list(range(start_date.year, end_date.year + 1))
    rows = []
    for y in years:
        rows += [
            {"ds": date(y,1,1), "type":"holiday", "weight":1.0},
            {"ds": date(y,5,1), "type":"holiday", "weight":1.0},
            {"ds": date(y,12,25), "type":"holiday", "weight":1.0},
            {"ds": date(y,12,26), "type":"holiday", "weight":1.0},
        ]
        easter = _easter_sunday(y)
        rows += [
            {"ds": easter - timedelta(days=2), "type":"holiday", "weight":1.0},
            {"ds": easter + timedelta(days=1), "type":"holiday", "weight":1.0},
        ]
        bf = _last_friday_of_november(y)
        rows += [
            {"ds": bf, "type":"event", "weight":0.6},
            {"ds": bf + timedelta(days=4), "type":"event", "weight":0.6},
        ]
    exo = pd.DataFrame(rows)
    exo["ds"] = pd.to_datetime(exo["ds"])
    return exo[(exo["ds"] >= start_date) & (exo["ds"] <= end_date)].reset_index(drop=True)

def build_exogenous_calendar(incoming: pd.DataFrame, horizon_days: int):
    start = incoming["Date"].min() - pd.Timedelta(days=365)
    end = incoming["Date"].max() + pd.Timedelta(days=horizon_days)
    exo = build_eu_core_holidays(start, end)
    cal = pd.DataFrame({"ds": pd.date_range(start, end, freq="D")})
    exo_daily = cal.merge(exo, on="ds", how="left")
    exo_daily["is_holiday"] = (exo_daily["type"] == "holiday").astype(int)
    exo_daily["is_event"] = (exo_daily["type"] == "event").astype(int)
    exo_daily["weight_h"] = np.where(exo_daily["is_holiday"] == 1, exo_daily["weight"].fillna(1.0), 0.0)
    exo_daily["weight_e"] = np.where(exo_daily["is_event"] == 1, exo_daily["weight"].fillna(1.0), 0.0)
    exo_daily = exo_daily[["ds","is_holiday","is_event","weight_h","weight_e"]]
    exo_daily["month"] = exo_daily["ds"].dt.to_period("M")
    exo_monthly = (exo_daily.groupby("month", as_index=False)
                   .agg(hol_count=("is_holiday","sum"),
                        evt_count=("is_event","sum"),
                        hol_weight_sum=("weight_h","sum"),
                        evt_weight_sum=("weight_e","sum")))
    exo_monthly["month"] = pd.PeriodIndex(exo_monthly["month"], freq="M")
    return exo_daily, exo_monthly

def prepare_monthly_exog(exo_monthly: pd.DataFrame, ts_index: pd.PeriodIndex,
                         outage_monthly: Optional[pd.DataFrame] = None,
                         department_id: Optional[str] = None) -> pd.DataFrame:
    ex = exo_monthly.copy()
    ex["month"] = pd.PeriodIndex(ex["month"], freq="M")
    ex = ex.set_index("month").reindex(ts_index).fillna(0.0)
    if outage_monthly is not None and department_id is not None:
        om = outage_monthly[outage_monthly["department_id"].astype(str) == str(department_id)].copy()
        if not om.empty:
            om = om.set_index("month").reindex(ts_index).fillna(0.0)
            ex["outage_tickets"] = om["outage_tickets"].astype(float).values
        else:
            ex["outage_tickets"] = 0.0
    else:
        ex["outage_tickets"] = 0.0
    return ex[["hol_count","evt_count","hol_weight_sum","evt_weight_sum","outage_tickets"]]

# ---------- Structural breaks + stability ----------
def detect_structural_break(ts_m: pd.Series, window: int = 6, min_total_months: int = 18, z_thresh: float = 2.5) -> Optional[pd.Period]:
    ts_m = ts_m.dropna()
    if len(ts_m) < max(min_total_months, 2*window + 1):
        return None
    last = ts_m.iloc[-window:]
    prev = ts_m.iloc[-2*window:-window]
    mu1, mu0 = float(last.mean()), float(prev.mean())
    s1, s0 = float(last.std(ddof=1)), float(prev.std(ddof=1))
    pooled = math.sqrt(max(1e-9, (s1*s1 + s0*s0) / 2.0))
    z = abs(mu1 - mu0) / pooled if pooled > 0 else 0.0
    return ts_m.index[-window] if z >= z_thresh else None

def apply_break_window(ts_m: pd.Series, break_start: Optional[pd.Period], min_train_months: int = 12) -> pd.Series:
    if break_start is None:
        return ts_m
    ts2 = ts_m.loc[break_start:]
    return ts2 if len(ts2) >= min_train_months else ts_m

def classify_department_series(ts_m: pd.Series) -> Dict[str, object]:
    ts_m = ts_m.dropna()
    n = int(len(ts_m))
    zeros_share = float((ts_m <= 0).mean()) if n > 0 else 1.0
    mean = float(ts_m.mean()) if n > 0 else 0.0
    std = float(ts_m.std(ddof=1)) if n > 1 else 0.0
    cv = float(std / mean) if mean > 0 else np.inf
    brk = detect_structural_break(ts_m, window=6, min_total_months=18, z_thresh=STAB["broken_z_thresh"])
    reasons = []
    if n < STAB["low_data_min_months"]:
        label = "LowData"; reasons.append(f"months<{STAB['low_data_min_months']}")
    else:
        label = "Stable"
        if zeros_share >= STAB["zero_share_thresh"]:
            label = "Volatile"; reasons.append(f"zero_share>={STAB['zero_share_thresh']}")
        if cv >= STAB["volatility_cv_thresh"]:
            label = "Volatile"; reasons.append(f"cv>={STAB['volatility_cv_thresh']}")
        if brk is not None:
            label = "Broken"; reasons.append(f"struct_break@{brk}")
    return {"label": label, "months": n, "zeros_share": zeros_share, "cv": cv,
            "break_start": str(brk) if brk is not None else None,
            "reasons": "; ".join(reasons) if reasons else None}

# ---------- Models ----------
def fit_prophet_monthly_log_with_pi(ts_m: pd.Series, exo_m: Optional[pd.DataFrame] = None, changepoint_prior_scale: float = 0.15):
    if Prophet is None or len(ts_m) < 6:
        return None, None, None
    cap = compute_dynamic_cap(ts_m)
    y = np.log1p(ts_m.values)
    dfp = pd.DataFrame({"ds": ts_m.index.to_timestamp(), "y": y})
    if exo_m is not None:
        dfp = dfp.join(exo_m, on=ts_m.index).reset_index(drop=True)
    m = Prophet(weekly_seasonality=False, yearly_seasonality=True, daily_seasonality=False,
                interval_width=(1.0 - PI_ALPHA), changepoint_prior_scale=changepoint_prior_scale)
    if exo_m is not None:
        for col in exo_m.columns:
            m.add_regressor(col)
    m.fit(dfp)

    def _future(h_months: int):
        future = m.make_future_dataframe(periods=h_months, freq="MS")
        if exo_m is not None:
            idx_future = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
            pad = pd.concat([exo_m.iloc[[-1]]] * h_months, ignore_index=True)
            pad.index = idx_future
            ex_full = pd.concat([exo_m, pad], axis=0)
            for col in exo_m.columns:
                future[col] = ex_full[col].values
        return future

    def f_mean(h_months: int):
        pred = m.predict(_future(h_months))
        pred.index = pd.PeriodIndex(pred["ds"], freq="M")
        mu_log = pred["yhat"].iloc[-h_months:].values
        return pd.Series(clamp_nonneg(expm1_safe(mu_log, cap_original=cap)), index=pred.index[-h_months:])

    def f_pi(h_months: int):
        pred = m.predict(_future(h_months))
        pred.index = pd.PeriodIndex(pred["ds"], freq="M")
        idx = pred.index[-h_months:]
        mu_log = pred["yhat"].iloc[-h_months:].values
        lo_log = pred["yhat_lower"].iloc[-h_months:].values
        hi_log = pred["yhat_upper"].iloc[-h_months:].values
        mean = clamp_nonneg(expm1_safe(mu_log, cap_original=cap))
        lo = clamp_nonneg(expm1_safe(lo_log, cap_original=cap))
        hi = clamp_nonneg(expm1_safe(hi_log, cap_original=cap))
        return pd.Series(mean, index=idx), pd.Series(np.minimum(lo,hi), index=idx), pd.Series(np.maximum(lo,hi), index=idx)

    return m, f_mean, f_pi

def fit_arima_monthly_log_with_pi(ts_m: pd.Series, exo_m: Optional[pd.DataFrame] = None):
    cap = compute_dynamic_cap(ts_m)
    y = np.log1p(ts_m)
    best_aic, best_model, best_exog, best_order = np.inf, None, None, None
    pqs = [0,1]
    seasonal = len(ts_m) >= 12
    PsQs = [0] if seasonal else [0]
    for p in pqs:
        for d in ([1] if len(ts_m) < 24 else [0,1]):
            for q in pqs:
                for P in PsQs:
                    for D in ([0,1] if seasonal else [0]):
                        for Q in PsQs:
                            try:
                                model = SARIMAX(
                                    y,
                                    order=(p,d,q),
                                    seasonal_order=(P,D,Q,12 if seasonal else 0),
                                    exog=(exo_m.values if exo_m is not None else None),
                                    enforce_stationarity=False,
                                    enforce_invertibility=False
                                ).fit(disp=False)
                                # Governance: reject non-converged
                                if hasattr(model, "mle_retvals"):
                                    if not bool(model.mle_retvals.get("converged", True)):
                                        continue
                                if model.aic < best_aic:
                                    best_aic, best_model, best_exog = model.aic, model, exo_m
                                    best_order = ((p,d,q),(P,D,Q,12 if seasonal else 0))
                            except Exception:
                                continue

    def _future_exog(h_months: int, idx: pd.PeriodIndex):
        if best_exog is None:
            return None
        pad = pd.concat([best_exog.iloc[[-1]]] * h_months, ignore_index=True)
        pad.index = idx
        return pad.values

    def f_mean(h_months: int):
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
        if best_model is None:
            mu = np.full(h_months, float(np.nanmean(y.values)))
            return pd.Series(clamp_nonneg(expm1_safe(mu, cap_original=cap)), index=idx)
        fc = best_model.get_forecast(h_months, exog=_future_exog(h_months, idx))
        mu_log = fc.predicted_mean.values
        return pd.Series(clamp_nonneg(expm1_safe(mu_log, cap_original=cap)), index=idx)

    def f_pi(h_months: int):
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
        if best_model is None:
            mu_log = np.full(h_months, float(np.nanmean(y.values)))
            std_log = np.full(h_months, float(np.nanstd(y.values) if np.isfinite(np.nanstd(y.values)) else 0.5))
            mean, lo, hi = compute_pi_from_log_mean_std(mu_log, std_log, cap=cap, alpha=PI_ALPHA)
            return pd.Series(mean,index=idx), pd.Series(lo,index=idx), pd.Series(hi,index=idx)
        fc = best_model.get_forecast(h_months, exog=_future_exog(h_months, idx))
        mu_log = fc.predicted_mean.values
        ci = fc.conf_int(alpha=PI_ALPHA)
        lo_log = ci.iloc[:,0].values
        hi_log = ci.iloc[:,1].values
        mean = clamp_nonneg(expm1_safe(mu_log, cap_original=cap))
        lo = clamp_nonneg(expm1_safe(lo_log, cap_original=cap))
        hi = clamp_nonneg(expm1_safe(hi_log, cap_original=cap))
        return pd.Series(mean,index=idx), pd.Series(np.minimum(lo,hi),index=idx), pd.Series(np.maximum(lo,hi),index=idx)

    return best_model, f_mean, f_pi, best_order

def fit_ets_monthly_log_with_pi(ts_m: pd.Series):
    cap = compute_dynamic_cap(ts_m)
    y_log = np.log1p(ts_m)
    seasonal = 12 if len(ts_m) >= 24 else None
    model = ExponentialSmoothing(y_log, trend="add", seasonal=("add" if seasonal else None), seasonal_periods=seasonal).fit()
    resid = np.array(y_log.values, dtype=float) - np.array(model.fittedvalues, dtype=float)
    sigma = float(np.nanstd(resid)) if np.isfinite(np.nanstd(resid)) else 0.5
    def f_mean(h_months: int):
        vals_log = np.array(model.forecast(h_months), dtype=float)
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
        return pd.Series(clamp_nonneg(expm1_safe(vals_log, cap_original=cap)), index=idx)
    def f_pi(h_months: int):
        vals_log = np.array(model.forecast(h_months), dtype=float)
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq="M")
        mean, lo, hi = compute_pi_from_log_mean_std(vals_log, np.full(h_months, sigma), cap=cap, alpha=PI_ALPHA)
        return pd.Series(mean,index=idx), pd.Series(lo,index=idx), pd.Series(hi,index=idx)
    return model, f_mean, f_pi

# ---------- CV + blending ----------
def rolling_cv_monthly_adaptive(ts_m: pd.Series, exo_monthly: Optional[pd.DataFrame] = None) -> Optional[Dict[str, float]]:
    n = len(ts_m)
    if n < 9:
        return None
    h = 3 if n >= 15 else 1
    min_train = max(12, n - (h + 2))
    splits = []
    for start in range(min_train, n - h + 1):
        train = ts_m.iloc[:start]
        test = ts_m.iloc[start:start+h]
        ex_train = exo_monthly.iloc[:start] if exo_monthly is not None else None
        metrics = {}
        mp, fp_mean, _ = fit_prophet_monthly_log_with_pi(train, ex_train, changepoint_prior_scale=0.2)
        if fp_mean is not None:
            try: metrics["Prophet"] = smape(test.values, fp_mean(h).values[:h])
            except Exception: metrics["Prophet"] = 200.0
        try:
            _, fa_mean, _, _ = fit_arima_monthly_log_with_pi(train, ex_train)
            metrics["ARIMA"] = smape(test.values, fa_mean(h).values[:h])
        except Exception:
            metrics["ARIMA"] = 200.0
        try:
            _, fe_mean, _ = fit_ets_monthly_log_with_pi(train)
            metrics["ETS"] = smape(test.values, fe_mean(h).values[:h])
        except Exception:
            metrics["ETS"] = 200.0
        splits.append(metrics)
    return pd.DataFrame(splits).mean().to_dict()

def select_or_blend_forecasts(fc_dict: Dict[str, pd.Series], cv_scores: Dict[str, float], blend: bool = True):
    if not fc_dict:
        raise ValueError("No forecast candidates.")
    scores = {k: float(v) for k,v in (cv_scores or {}).items() if v is not None and np.isfinite(v)}
    if (not blend) or len(scores) == 0:
        k = next(iter(fc_dict.keys()))
        return fc_dict[k], {"winner": k, "weights": {k: 1.0}}
    inv = {k: (1.0/v if v > 0 else 0.0) for k,v in scores.items()}
    total = sum(inv.values())
    if total <= 0:
        k = min(scores, key=scores.get)
        return fc_dict[k], {"winner": k, "weights": {k: 1.0}}
    w = {k: inv[k]/total for k in inv}
    idx = None
    for s in fc_dict.values():
        idx = s.index if idx is None else idx.union(s.index)
    blended = sum(w.get(k,0.0) * fc_dict[k].reindex(idx).fillna(0.0) for k in fc_dict)
    winner = min(scores, key=scores.get)
    return blended, {"winner": winner, "weights": w}

def select_or_blend_forecasts_with_pi(fc_mean, fc_pi, cv_scores, blend=True):
    mean_s, meta = select_or_blend_forecasts(fc_mean, cv_scores, blend=blend)
    if not fc_pi:
        idx = mean_s.index
        return mean_s, pd.Series(np.nan, index=idx), pd.Series(np.nan, index=idx), meta
    z = _z_from_alpha(PI_ALPHA)
    idx = None
    for s in fc_mean.values():
        idx = s.index if idx is None else idx.union(s.index)
    var_log = np.zeros(len(idx), dtype=float)
    weights = meta.get("weights", {})
    for k, w in weights.items():
        if k not in fc_pi: continue
        mu_s, lo_s, hi_s = fc_pi[k]
        lo = lo_s.reindex(idx).astype(float).values
        hi = hi_s.reindex(idx).astype(float).values
        lo_log = np.log1p(np.clip(lo, 0, None))
        hi_log = np.log1p(np.clip(hi, 0, None))
        sigma = (hi_log - lo_log) / (2.0*z)
        sigma = np.clip(sigma, 1e-8, None)
        var_log += (float(w)**2) * (sigma**2)
    std_log = np.sqrt(np.clip(var_log, 1e-10, None))
    mu_bl = mean_s.reindex(idx).astype(float).values
    mu_log_bl = np.log1p(np.clip(mu_bl, 0, None))
    cap = float(np.nanmax(mu_bl) * 6.0) if np.isfinite(np.nanmax(mu_bl)) and np.nanmax(mu_bl) > 0 else np.inf
    mean, lo, hi = compute_pi_from_log_mean_std(mu_log_bl, std_log, cap=cap, alpha=PI_ALPHA)
    return pd.Series(mean, index=idx), pd.Series(lo, index=idx), pd.Series(hi, index=idx), meta

# ---------- Monthly forecast by department (adaptive) ----------
def forecast_monthly_by_department_v10(incoming_clean: pd.DataFrame, exo_monthly: pd.DataFrame, outage_monthly: Optional[pd.DataFrame]):
    out_rows, st_rows = [], []
    for dept, g in incoming_clean.groupby("department_id"):
        g = clean_outliers_daily(g.sort_values("Date"))
        gm = g.assign(month=g["Date"].dt.to_period("M")).groupby("month")["ticket_total"].sum().sort_index()
        gm.index = pd.PeriodIndex(gm.index, freq="M")
        if len(gm) == 0: 
            continue
        ts_full = winsorize_monthly(gm)
        st = classify_department_series(ts_full)
        st_rows.append({"department_id": str(dept), **st})

        brk = detect_structural_break(ts_full, z_thresh=STAB["broken_z_thresh"])
        ts_train = apply_break_window(ts_full, brk, min_train_months=ACCURACY_MIN_TRAIN_MONTHS)
        ex_train = prepare_monthly_exog(exo_monthly, ts_train.index, outage_monthly, department_id=str(dept))

        label = st["label"]
        use_blend = (label == "Stable")
        prophet_cp = 0.15 if label == "Stable" else (0.25 if label == "Volatile" else 0.35)
        allow_arima = (label == "Stable")
        allow_ets = True

        fc_mean, fc_pi = {}, {}
        models_used = []

        mp, fp_mean, fp_pi = fit_prophet_monthly_log_with_pi(ts_train, ex_train, changepoint_prior_scale=prophet_cp)
        if fp_mean is not None:
            fc_mean["Prophet"] = fp_mean(H_MONTHS); models_used.append("Prophet")
            if ENABLE_MONTHLY_PI and fp_pi is not None:
                fc_pi["Prophet"] = fp_pi(H_MONTHS)

        arima_order = None
        if allow_arima:
            try:
                _, fa_mean, fa_pi, order = fit_arima_monthly_log_with_pi(ts_train, ex_train)
                if fa_mean is not None:
                    fc_mean["ARIMA"] = fa_mean(H_MONTHS); models_used.append("ARIMA")
                    arima_order = str(order)
                    if ENABLE_MONTHLY_PI and fa_pi is not None:
                        fc_pi["ARIMA"] = fa_pi(H_MONTHS)
            except Exception:
                pass

        try:
            _, fe_mean, fe_pi = fit_ets_monthly_log_with_pi(ts_train)
            fc_mean["ETS"] = fe_mean(H_MONTHS); models_used.append("ETS")
            if ENABLE_MONTHLY_PI and fe_pi is not None:
                fc_pi["ETS"] = fe_pi(H_MONTHS)
        except Exception:
            pass

        if not fc_mean:
            idx = pd.period_range(ts_train.index[-1] + 1, periods=H_MONTHS, freq="M")
            val = max(0.0, float(ts_train.mean()))
            fc_mean["NaiveMean"] = pd.Series([val]*H_MONTHS, index=idx)
            models_used.append("NaiveMean")

        cv = rolling_cv_monthly_adaptive(ts_train, ex_train) or {}
        if ENABLE_MONTHLY_PI and len(fc_pi) >= 1:
            mean_s, p05, p95, meta = select_or_blend_forecasts_with_pi(fc_mean, fc_pi, cv_scores=cv, blend=use_blend)
        else:
            mean_s, meta = select_or_blend_forecasts(fc_mean, cv_scores=cv, blend=use_blend)
            p05 = pd.Series(np.nan, index=mean_s.index)
            p95 = pd.Series(np.nan, index=mean_s.index)

        for per in mean_s.index:
            out_rows.append({
                "department_id": str(dept),
                "month": per,
                "forecast_monthly_dept": float(mean_s.loc[per]),
                "forecast_p05_dept": float(p05.loc[per]) if per in p05.index else np.nan,
                "forecast_p95_dept": float(p95.loc[per]) if per in p95.index else np.nan,
                "stability_label": label,
                "blend": bool(use_blend),
                "prophet_cp": float(prophet_cp),
                "models_used": ",".join(models_used),
                "winner_model": meta.get("winner"),
                "arima_order": arima_order,
                "break_start": st.get("break_start"),
            })

    fc_dept = pd.DataFrame(out_rows)
    if not fc_dept.empty:
        fc_dept["month"] = pd.PeriodIndex(fc_dept["month"], freq="M")
    dept_stability = pd.DataFrame(st_rows)
    return fc_dept, dept_stability

# ---------- Language shares ----------
def compute_language_shares_rolling3(incoming: pd.DataFrame) -> pd.DataFrame:
    d = incoming.copy()
    d["Date"] = pd.to_datetime(d["Date"])
    d["month"] = d["Date"].dt.to_period("M")
    d["language"] = d["language"].astype(str).str.strip()
    d["language"] = np.where(d["language"].isin(SUPPORTED_LANGS), d["language"], "English")
    agg = (d.groupby(["department_id","month","language"], as_index=False)["ticket_total"].sum()
             .sort_values(["department_id","language","month"]))
    totals = agg.groupby(["department_id","month"], as_index=False)["ticket_total"].sum().rename(columns={"ticket_total":"dept_total"})
    agg = agg.merge(totals, on=["department_id","month"], how="left")
    agg["share"] = np.where(agg["dept_total"] > 0, agg["ticket_total"]/agg["dept_total"], 0.0)
    agg["share_roll3"] = (agg.groupby(["department_id","language"])["share"]
                            .rolling(3, min_periods=1).mean()
                            .reset_index(level=[0,1], drop=True))
    agg = agg[agg["language"].isin(SUPPORTED_LANGS)].copy()
    return agg[["department_id","month","language","share_roll3"]]

def apply_language_split(fc_dept: pd.DataFrame, shares: pd.DataFrame) -> pd.DataFrame:
    f = fc_dept.copy()
    f["month"] = pd.PeriodIndex(f["month"], freq="M")
    lang_df = pd.DataFrame({"language": SUPPORTED_LANGS})
    f_exp = f.merge(lang_df, how="cross")
    sh = shares.copy()
    sh["month"] = pd.PeriodIndex(sh["month"], freq="M")
    f_exp = f_exp.merge(sh, on=["department_id","month","language"], how="left")
    f_exp["share_roll3"] = f_exp["share_roll3"].fillna(0.0)
    sums = f_exp.groupby(["department_id","month"])["share_roll3"].transform("sum")
    f_exp["share_norm"] = np.where(sums > 0, f_exp["share_roll3"]/sums, 0.0)
    f_exp["forecast_monthly"] = f_exp["forecast_monthly_dept"].astype(float) * f_exp["share_norm"].astype(float)
    f_exp["forecast_p05"] = f_exp["forecast_p05_dept"].astype(float) * f_exp["share_norm"].astype(float)
    f_exp["forecast_p95"] = f_exp["forecast_p95_dept"].astype(float) * f_exp["share_norm"].astype(float)
    keep = ["department_id","language","month","forecast_monthly","forecast_p05","forecast_p95","share_norm",
            "stability_label","blend","winner_model","models_used","break_start"]
    return f_exp[keep]

# ---------- Backtest + bias recalibration ----------
def backtest_accuracy_dept(incoming: pd.DataFrame, exo_monthly: pd.DataFrame, outage_monthly: Optional[pd.DataFrame], mapping: pd.DataFrame) -> pd.DataFrame:
    d = apply_mapping(incoming.copy(), mapping)
    d["Date"] = pd.to_datetime(d["Date"])
    d["month"] = d["Date"].dt.to_period("M")
    out = []
    for dept, g in d.groupby("department_id"):
        gm = g.groupby("month")["ticket_total"].sum().sort_index()
        gm.index = pd.PeriodIndex(gm.index, freq="M")
        if len(gm) < (ACCURACY_MIN_TRAIN_MONTHS + ACCURACY_BACKTEST_MONTHS):
            continue
        eval_months = gm.index[-ACCURACY_BACKTEST_MONTHS:]
        preds, lo_s, hi_s, acts = [], [], [], []
        splits_done = 0
        for m in eval_months[::-1]:
            train_end = m - 1
            train = gm.loc[:train_end]
            if len(train) < ACCURACY_MIN_TRAIN_MONTHS: 
                continue
            st = classify_department_series(train)
            label = st["label"]
            brk = detect_structural_break(train, z_thresh=STAB["broken_z_thresh"])
            train2 = apply_break_window(train, brk, min_train_months=ACCURACY_MIN_TRAIN_MONTHS)
            ex_train = prepare_monthly_exog(exo_monthly, train2.index, outage_monthly, department_id=str(dept))
            use_blend = (label == "Stable")
            prophet_cp = 0.15 if label == "Stable" else (0.25 if label == "Volatile" else 0.35)
            allow_arima = (label == "Stable")

            fc_mean, fc_pi = {}, {}
            mp, fp_mean, fp_pi = fit_prophet_monthly_log_with_pi(train2, ex_train, changepoint_prior_scale=prophet_cp)
            if fp_mean is not None:
                fc_mean["Prophet"] = fp_mean(ACCURACY_HORIZON_MONTHS)
                if ENABLE_MONTHLY_PI and fp_pi is not None:
                    fc_pi["Prophet"] = fp_pi(ACCURACY_HORIZON_MONTHS)
            if allow_arima:
                try:
                    _, fa_mean, fa_pi, _ = fit_arima_monthly_log_with_pi(train2, ex_train)
                    fc_mean["ARIMA"] = fa_mean(ACCURACY_HORIZON_MONTHS)
                    if ENABLE_MONTHLY_PI and fa_pi is not None:
                        fc_pi["ARIMA"] = fa_pi(ACCURACY_HORIZON_MONTHS)
                except Exception:
                    pass
            try:
                _, fe_mean, fe_pi = fit_ets_monthly_log_with_pi(train2)
                fc_mean["ETS"] = fe_mean(ACCURACY_HORIZON_MONTHS)
                if ENABLE_MONTHLY_PI and fe_pi is not None:
                    fc_pi["ETS"] = fe_pi(ACCURACY_HORIZON_MONTHS)
            except Exception:
                pass
            if not fc_mean:
                continue
            cv = rolling_cv_monthly_adaptive(train2, ex_train) or {}
            if ENABLE_MONTHLY_PI and len(fc_pi) >= 1:
                mean_s, p05, p95, _ = select_or_blend_forecasts_with_pi(fc_mean, fc_pi, cv_scores=cv, blend=use_blend)
            else:
                mean_s, _ = select_or_blend_forecasts(fc_mean, cv_scores=cv, blend=use_blend)
                p05 = pd.Series(np.nan, index=mean_s.index)
                p95 = pd.Series(np.nan, index=mean_s.index)
            if m in mean_s.index:
                preds.append(float(mean_s.loc[m]))
                lo_s.append(float(p05.loc[m]) if m in p05.index else np.nan)
                hi_s.append(float(p95.loc[m]) if m in p95.index else np.nan)
                acts.append(float(gm.loc[m]))
                splits_done += 1
            if splits_done >= ACCURACY_MAX_SPLITS:
                break
        if len(acts) == 0:
            continue
        y_true = np.array(acts, dtype=float)
        y_pred = np.array(preds, dtype=float)
        sm = smape(y_true, y_pred)
        mae = float(np.nanmean(np.abs(y_pred - y_true)))
        bias = float(np.nanmean((y_pred - y_true) / np.where(y_true > 0, y_true, np.nan)) * 100.0)
        acc = float(np.nanmean((1 - (np.abs(y_pred - y_true) / np.where(y_true > 0, y_true, np.nan))) * 100.0))
        cov = coverage_95(y_true, np.array(lo_s, dtype=float), np.array(hi_s, dtype=float)) if ENABLE_MONTHLY_PI else np.nan
        out.append({"department_id": str(dept),
                    "vertical": g["vertical"].iloc[0] if "vertical" in g.columns else None,
                    "department_name": g["department_name"].iloc[0] if "department_name" in g.columns else None,
                    "Eval_Months": int(len(y_true)),
                    "sMAPE_%": float(sm),
                    "MAE": float(mae),
                    "Bias_%": float(bias),
                    "Accuracy_%": float(acc),
                    "PI_Coverage_95_%": float(cov)})
    df = pd.DataFrame(out)
    return df.sort_values(["vertical","department_id"]) if not df.empty else df

def apply_bias_recalibration(fc_dept: pd.DataFrame, acc_dept: pd.DataFrame) -> pd.DataFrame:
    df = fc_dept.copy()
    if acc_dept is None or acc_dept.empty:
        df["bias_correction_factor"] = 1.0
        return df
    a = acc_dept[["department_id","Bias_%"]].copy()
    a["department_id"] = a["department_id"].astype(str)
    df = df.merge(a, on="department_id", how="left")
    b = df["Bias_%"].astype(float)
    needs = b.abs() > BIAS_RECAL_THRESHOLD_PCT
    corr = np.where(needs, 1.0 / (1.0 + (b/100.0)), 1.0)
    corr = np.clip(corr, 0.7, 1.3)
    df["bias_correction_factor"] = corr
    for col in ["forecast_monthly_dept","forecast_p05_dept","forecast_p95_dept"]:
        df[col] = df[col].astype(float) * df["bias_correction_factor"].astype(float)
    return df.drop(columns=["Bias_%"], errors="ignore")

# ---------- Staffing (risk-based) ----------
def compute_staffing_volume_monthly_v10(fc_dept: pd.DataFrame, acc_dept: pd.DataFrame, mapping: pd.DataFrame, dept_stability: pd.DataFrame) -> pd.DataFrame:
    df = fc_dept.copy().merge(mapping, on="department_id", how="left")
    df = df.merge(dept_stability[["department_id","label"]].rename(columns={"label":"stability_label"}), on="department_id", how="left")
    if acc_dept is None or acc_dept.empty:
        df["Accuracy_%"] = np.nan
    else:
        df = df.merge(acc_dept[["department_id","Accuracy_%"]], on="department_id", how="left")

    def _uplift(label: str) -> float:
        if label == "Broken": return RISK_POLICY["broken_uplift_pct"]
        if label == "LowData": return RISK_POLICY["lowdata_uplift_pct"]
        if label == "Volatile": return RISK_POLICY["volatility_uplift_pct"]
        return 0.0

    def _critical_rule(acc, f, p95):
        if (acc is None) or (not np.isfinite(acc)): return p95
        if acc < RISK_POLICY["critical_low_acc_use_p95"]: return p95
        if acc < RISK_POLICY["critical_mid_acc_blend"]: return f + 0.5*(p95 - f)
        return f

    staffing_vals, uplifts = [], []
    for _, r in df.iterrows():
        f = float(r["forecast_monthly_dept"])
        p95 = float(r["forecast_p95_dept"]) if np.isfinite(r["forecast_p95_dept"]) else f
        acc = r.get("Accuracy_%", np.nan)
        vert = r.get("vertical", None)
        label = r.get("stability_label", "Stable")

        base = _critical_rule(acc, f, p95) if vert in CRITICAL_VERTICALS else f
        up = float(_uplift(label))
        staffing_vals.append(base * (1.0 + up/100.0))
        uplifts.append(up)

    df["risk_uplift_pct"] = uplifts
    df["staffing_volume_monthly"] = staffing_vals
    return df

# ---------- Weekend policy + DOW profile + daily allocation ----------
def get_weekend_service_policy(mapping: pd.DataFrame) -> pd.DataFrame:
    mp = mapping.copy()
    mp["vertical"] = mp.get("vertical", "Unmapped").astype(str).str.strip()
    mp["department_name"] = mp.get("department_name", "Unknown").astype(str).str.strip()
    payments_no_weekend_names = {"CA_PYAC","L2 Customer Support","Datatrans L2 Customer Support","Specialist - L2 Customer Support"}
    mp["weekend_service"] = True
    mp.loc[mp["vertical"].str.lower() == "partners", "weekend_service"] = False
    is_payments = mp["vertical"].str.lower() == "payments"
    mp.loc[is_payments & mp["department_name"].isin(payments_no_weekend_names), "weekend_service"] = False
    return mp[["department_id","vertical","department_name","weekend_service"]].drop_duplicates("department_id")

def compute_dept_dow_profile(incoming: pd.DataFrame, lookback_days: int = 180, min_obs: int = 30) -> pd.DataFrame:
    d = incoming.copy()
    d["Date"] = pd.to_datetime(d["Date"])
    maxd = d["Date"].max()
    d = d[d["Date"] >= (maxd - pd.Timedelta(days=lookback_days))].copy()
    d["dow"] = d["Date"].dt.weekday.astype(int)
    agg = d.groupby(["department_id","dow"], as_index=False)["ticket_total"].sum()
    totals = agg.groupby("department_id", as_index=False)["ticket_total"].sum().rename(columns={"ticket_total":"dept_total"})
    agg = agg.merge(totals, on="department_id", how="left")
    agg["weight_raw"] = np.where(agg["dept_total"] > 0, agg["ticket_total"]/agg["dept_total"], 0.0)
    depts = d["department_id"].astype(str).unique().tolist()
    grid = pd.MultiIndex.from_product([depts, list(range(7))], names=["department_id","dow"]).to_frame(index=False)
    prof = grid.merge(agg[["department_id","dow","weight_raw"]], on=["department_id","dow"], how="left").fillna(0.0)
    obs = d.groupby("department_id")["Date"].count().rename("n_obs").reset_index()
    prof = prof.merge(obs, on="department_id", how="left").fillna({"n_obs":0})
    def _fallback():
        w = np.zeros(7, dtype=float); w[:5] = 1.0/5.0
        return w
    out=[]
    for dept, g in prof.groupby("department_id"):
        n = int(g["n_obs"].iloc[0])
        if n < min_obs or float(g["weight_raw"].sum()) <= 0:
            w = _fallback()
        else:
            w = g.sort_values("dow")["weight_raw"].values.astype(float)
            s = float(w.sum()); w = w/s if s>0 else _fallback()
        for dow, wv in enumerate(w):
            out.append({"department_id": str(dept), "dow": int(dow), "dow_weight": float(wv)})
    return pd.DataFrame(out)

def build_daily_plan_from_monthly_staffing(staff_m: pd.DataFrame, incoming_hist: pd.DataFrame,
                                           mapping_policy: pd.DataFrame, dept_dow_profile: pd.DataFrame,
                                           horizon_days: int) -> pd.DataFrame:
    last_date = incoming_hist["Date"].max()
    start = last_date + pd.Timedelta(days=1)
    end = start + pd.Timedelta(days=horizon_days - 1)
    wknd = dict(zip(mapping_policy["department_id"].astype(str), mapping_policy["weekend_service"].astype(bool)))
    prof = dept_dow_profile.copy()
    prof_key = {(r["department_id"], int(r["dow"])): float(r["dow_weight"]) for _, r in prof.iterrows()}
    rows=[]
    for _, r in staff_m.iterrows():
        dept = str(r["department_id"])
        m = r["month"]; m = m if isinstance(m, pd.Period) else pd.Period(m, freq="M")
        month_days = pd.date_range(m.start_time, m.end_time, freq="D")
        month_days = month_days[(month_days >= start) & (month_days <= end)]
        if len(month_days)==0: 
            continue
        weekend_service = bool(wknd.get(dept, True))
        w=[]
        for dday in month_days:
            dow = int(dday.weekday())
            if (not weekend_service) and (dow>=5): w.append(0.0)
            else: w.append(float(prof_key.get((dept, dow), 0.0)))
        w=np.array(w, dtype=float); s=float(w.sum())
        if s<=0:
            allowed = np.array([(dday.weekday() < 5) if (not weekend_service) else True for dday in month_days], dtype=bool)
            w=allowed.astype(float); s=float(w.sum()) if float(w.sum())>0 else 1.0
        w=w/s
        monthly_total=float(r["staffing_volume_monthly"])
        daily_alloc=monthly_total*w
        for dday, val in zip(month_days, daily_alloc):
            rows.append({"Date": dday, "department_id": dept,
                         "vertical": r.get("vertical"), "department_name": r.get("department_name"),
                         "weekend_service": weekend_service,
                         "Staffing_Daily_Tickets": float(max(0.0, val))})
    return pd.DataFrame(rows)


## 4) Run + Export

In [4]:
incoming = load_incoming(INCOMING_SOURCE_PATH, INCOMING_SHEET)
mapping = load_dept_map(DEPT_MAP_PATH, DEPT_MAP_SHEET)
incoming = apply_mapping(incoming, mapping)

dept_prod, dept_dow_prod = load_productivity(PRODUCTIVITY_PATH)
outage_monthly = load_case_reason_outage_monthly(CASE_REASON_PATH)

exo_daily, exo_monthly = build_exogenous_calendar(incoming, horizon_days=DAILY_HORIZON_DAYS)

fc_dept, dept_stability = forecast_monthly_by_department_v10(incoming, exo_monthly, outage_monthly)

acc_dept = backtest_accuracy_dept(incoming, exo_monthly, outage_monthly, mapping) if ENABLE_DEPT_ACCURACY_TABLE else pd.DataFrame()
fc_dept_adj = apply_bias_recalibration(fc_dept, acc_dept)

shares = compute_language_shares_rolling3(incoming)
fc_dept_lang = apply_language_split(fc_dept_adj, shares).merge(mapping, on="department_id", how="left")

staff_m_dept = compute_staffing_volume_monthly_v10(fc_dept_adj, acc_dept, mapping, dept_stability)

dept_dow_profile = compute_dept_dow_profile(incoming, lookback_days=DOW_LOOKBACK_DAYS, min_obs=DOW_MIN_OBS)
dept_policy = get_weekend_service_policy(mapping)

daily_staff = build_daily_plan_from_monthly_staffing(staff_m_dept, incoming, dept_policy, dept_dow_profile, horizon_days=DAILY_HORIZON_DAYS)

daily_staff["dow"] = pd.to_datetime(daily_staff["Date"]).dt.weekday.astype(int)
daily_staff = daily_staff.merge(dept_dow_prod, on=["department_id","dow"], how="left")
daily_staff = daily_staff.merge(dept_prod, on="department_id", how="left")
daily_staff["prod_used"] = daily_staff["avg_tickets_per_agent_day_dow"].fillna(daily_staff["avg_tickets_per_agent_day"])
daily_staff["Capacity_FTE_per_day"] = np.where(
    pd.to_numeric(daily_staff["prod_used"], errors="coerce").fillna(0.0) > 0,
    daily_staff["Staffing_Daily_Tickets"] / pd.to_numeric(daily_staff["prod_used"], errors="coerce"),
    np.nan
)

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as w:
    dept_stability.merge(mapping, on="department_id", how="left").sort_values(["vertical","department_id"]).to_excel(w, "dept_stability", index=False)
    fc_dept_adj.merge(mapping, on="department_id", how="left").sort_values(["vertical","department_id","month"]).to_excel(w, "forecast_dept_monthly", index=False)
    fc_dept_lang.sort_values(["vertical","department_id","language","month"]).to_excel(w, "forecast_dept_lang_monthly", index=False)
    staff_m_dept.sort_values(["vertical","department_id","month"]).to_excel(w, "staffing_monthly_dept", index=False)
    shares.sort_values(["department_id","language","month"]).to_excel(w, "language_share_roll3", index=False)
    dept_dow_profile.sort_values(["department_id","dow"]).to_excel(w, "dow_profile_dept", index=False)
    dept_policy.sort_values(["vertical","department_id"]).to_excel(w, "weekend_service_policy", index=False)
    daily_staff.sort_values(["vertical","department_id","Date"]).to_excel(w, "daily_capacity_plan_90d", index=False)
    if acc_dept is not None and not acc_dept.empty:
        acc_dept.sort_values(["vertical","department_id"]).to_excel(w, "accuracy_dept_monthly", index=False)

print("✅ v10 export complete:", OUTPUT_XLSX)
display(dept_stability.head(10))
display(acc_dept.head(10))
display(staff_m_dept.head(10))


08:32:39 - cmdstanpy - INFO - Chain [1] start processing
08:32:39 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\anto-\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\anto-\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\anto-\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\anto-\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning

✅ v10 export complete: C:\Projects\Capacity_forecast_2026\outputs\capacity_forecast_hybrid_v10.xlsx


,department_id,label,months,zeros_share,cv,break_start,reasons
0,1,Stable,32,0.0,0.428414,None,None
1,10,Stable,28,0.0,0.345793,None,None
2,11,Stable,30,0.0,0.447863,None,None
3,12,Stable,32,0.0,0.300262,None,None
4,13,Stable,32,0.0,0.323964,None,None
5,14,Stable,32,0.0,0.360058,None,None
6,15,Stable,32,0.0,0.500638,None,None
7,16,Stable,32,0.0,0.288934,None,None
8,18,Stable,32,0.0,0.415363,None,None
9,2,Stable,32,0.0,0.484091,None,None


,department_id,vertical,department_name,Eval_Months,sMAPE_%,MAE,Bias_%,Accuracy_%,PI_Coverage_95_%
1,10,Hospitality,CS_PMSP_PREM_L2,9,36.577224,238.719458,47.910222,49.238177,44.444444
2,11,Hospitality,CS_PMSP_CLOUD_L2,9,84.583399,614.431280,186.641608,-87.073179,11.111111
11,23,Hospitality,CS_PMSP_FRANCE,9,98.978653,1523.117829,252.356197,-152.356197,33.333333
13,4,Hospitality,CS_PMSP_DIST,8,170.590241,2823.137670,1320.762367,-1270.762367,0.000000
15,5,Hospitality,CS_PMSP_INTEG,9,61.636129,1128.617840,101.887273,-1.887273,11.111111
16,6,Hospitality,CS_PMSP_KEY,9,195.297928,1147.616098,7679.903219,-7668.767279,0.000000
18,7,Hospitality,CS_PMSH_L1,9,34.227857,493.902716,19.167391,58.891800,22.222222
19,8,Hospitality,CS_PMSP_CLOUD_L1,9,58.766115,520.326517,69.465019,10.287931,11.111111
20,9,Hospitality,CS_PMSP_PREM_L1,9,96.054831,3316.549985,241.275634,-148.869520,66.666667
3,12,Partners,CS_PART_APAC,9,22.546829,118.646033,19.678502,73.264573,44.444444


,department_id,month,forecast_monthly_dept,forecast_p05_dept,forecast_p95_dept,stability_label_x,blend,prophet_cp,models_used,winner_model,arima_order,break_start,bias_correction_factor,department_name,vertical,stability_label_y,Accuracy_%,risk_uplift_pct,staffing_volume_monthly
0,1,2026-03,2281.555603,1258.268458,4136.568574,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,4136.568574
1,1,2026-04,10146.950482,5013.465547,20536.079117,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,20536.079117
2,1,2026-05,7376.296503,3639.538669,14948.890496,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,14948.890496
3,1,2026-06,3972.803646,1958.782954,8056.900703,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,8056.900703
4,1,2026-07,7093.381672,3494.712842,14397.025544,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,14397.025544
5,1,2026-08,3516.440495,1728.765279,7151.960083,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,7151.960083
6,1,2026-09,3311.831095,1623.974141,6753.184967,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,6753.184967
7,1,2026-10,3259.050817,1593.266803,6665.671827,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,6665.671827
8,1,2026-11,1463.867328,712.008443,3008.886150,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,3008.886150
9,1,2026-12,3273.246200,1581.235303,6775.002963,Stable,True,0.15,"Prophet,ARIMA,ETS",ARIMA,"((0, 0, 1), (0, 1, 0, 12))",None,0.7,CS_GT3C_EU,Payments,Stable,-28.144521,0.0,6775.002963
